# Using `generate_events_async` and Streaming

This guide will teach you how to use the `LLMRails.generate_events_async` method and stream the bot response. This is useful when your rails need to process other events, in addition to user and bot messages. 

**Important**: the streaming option does not work with the synchronous method `LLMRails.generate_events`.

**Note**: this guide assumes you have successfully installed NeMo Guardrails and the OpenAI package. If not, please refer to the [Hello World](../../docs/getting_started/1_hello_world) guide.

In [1]:
import os

# Setting the TOKENIZERS_PARALLELISM to get rid of the forking warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Step 1: create a config 

Let's create a simple config:

In [2]:
from nemoguardrails import RailsConfig, LLMRails

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: gpt-4

streaming: True
"""

config = RailsConfig.from_content(yaml_content=YAML_CONFIG)
app = LLMRails(config)

Next, we need to create a streaming handler and register it in the current async context by setting the value of the AsyncIO context variable `streaming_handler_var`, create a demo task that prints the tokens and make the `generate_events_async` call:

In [3]:
import asyncio 
from nemoguardrails.streaming import StreamingHandler
from nemoguardrails.context import streaming_handler_var

# Create the streaming handler and register it.
streaming_handler = StreamingHandler()
streaming_handler_var.set(streaming_handler)

# For demo purposes, create a task that prints the tokens.
async def process_tokens():
    async for chunk in streaming_handler:
        print(f"CHUNK: {chunk}")

asyncio.create_task(process_tokens())

# Call the events-based API.
events = [{
  "type": "UtteranceUserActionFinished",
  "final_transcript": "Hello! How are you?"
}]

new_events = await app.generate_events_async(events)
print(f"There were {len(new_events)} new events.")

CHUNK: Hello
CHUNK: !
CHUNK:  As
CHUNK:  an
CHUNK:  AI
CHUNK: ,
CHUNK:  I
CHUNK:  don
CHUNK: 't
CHUNK:  have
CHUNK:  feelings
CHUNK: ,
CHUNK:  but
CHUNK:  I
CHUNK: 'm
CHUNK:  here
CHUNK:  and
CHUNK:  ready
CHUNK:  to
CHUNK:  assist
CHUNK:  you
CHUNK: .
CHUNK:  How
CHUNK:  can
CHUNK:  I
CHUNK:  help
CHUNK:  you
CHUNK:  today
CHUNK: ?
There were 12 new events.


As expected, the tokens were printed as they were generated, and at the end we get the complete list of events that were generated. For more details on the structure of the events, check out the [Event-based API Guide](../../docs/user_guides/advanced/event-based-api.md).